In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
import unidecode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# get all link auchan

In [2]:
def get_all_link_auchan():
    
    offer_page = 'https://www.auchan.hu/ajanlatok'
    
    page = urllib.request.urlopen(offer_page)
    soup = bs(page)
    divs = soup.body.findAll('div', {'class' : ['box submenu']})
        
    all_link = []
    offer_page = 'https://www.auchan.hu/ajanlatok'
    for div in divs:
        for a in div.find_all('a', href=True):
            link = a['href']
            #print(a)
            #print(f'original link:  {link}')
            '''
            if link.endswith('-'):
                link = link[:-1] + '/200'
                all_link.append(link)
            else:
                link = link + '/200'
                all_link.append(link)
            '''
            if link.count('/') == 4:
                link = link + '/mind/500'
                all_link.append(link)
                print(link)
            else:
                link = link + '/300'

    return all_link

In [3]:
all_link = get_all_link_auchan()

https://www.auchan.hu/ajanlat/otthon-haztartas-kert-auto/mind/500
https://www.auchan.hu/ajanlat/sport-jatek/mind/500
https://www.auchan.hu/ajanlat/friss-elelmiszer/mind/500
https://www.auchan.hu/ajanlat/italok/mind/500
https://www.auchan.hu/ajanlat/fagyasztott-elelmiszerek/mind/500
https://www.auchan.hu/ajanlat/tartos-elelmiszerek/mind/500
https://www.auchan.hu/ajanlat/illatszer-szepsegapolas/mind/500
https://www.auchan.hu/ajanlat/allatvilag/mind/500
https://www.auchan.hu/ajanlat/vegyi-aruk-haztartas/mind/500
https://www.auchan.hu/ajanlat/babavilag/mind/500
https://www.auchan.hu/ajanlat/muszaki-cikkek/mind/500
https://www.auchan.hu/ajanlat/ruhazat-1/mind/500
https://www.auchan.hu/ajanlat/grill-ajanlatok/mind/500


In [4]:
def get_all_offer_auchan(all_link):
    
    def find_nth_occurrence(string, char, occurrence):
        val = -1
        for i in range(0, occurrence):
            val = string.find(char, val + 1)
        return val
    
    counter = 1
    
    all_items = []
    
    for url in all_link:
        
        print(f'crawl url: {url} done {counter} from {len(all_link)}')
        counter = counter + 1
        
        page = urllib.request.urlopen(url)
        soup = bs(page)

        divs = soup.body.findAll('div', {'class' : ['row prodList']})
        
        for div in divs:
            prods = div.findAll('div', {'class' : ['box p10px prodItem']})
            #print(prods)
            if len(prods) > 0:
                for prod in prods:

                    item_dict = {}

                    item_dict['itemId'] = np.nan

                    #print(prod.find('a', {'class' : ['title red']}).get_text())
                    item_dict['itemName'] = prod.find('a', {'class' : ['title red']}).get_text()
                    item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()

                    #print(prod.find('strong', {'class' : ['red']}).get_text())
                    item_dict['price'] = prod.find('strong', {'class' : ['red']}).get_text()
                    
                    #print(prod.find('p').get_text().strip())
                    
                    #print(item_dict)
                    
                    item_dict['source'] = url[find_nth_occurrence(url, '/', 4)+1:find_nth_occurrence(url, '/', 5)]
                    
                    measure = prod.find('p')
                    
                    if measure != None:
                        measure_cut = measure.get_text().strip('\n').strip('\t').replace('\n','').strip()
                        measure_final = measure_cut[:measure_cut.find(' ')].replace('Kiszerelés:', '')
                        sales_from_string = measure_cut[measure_cut.find(' '):]
                        sales_from_pattern = r'(?P<group_1>[\d]{4}.[\d]{2}.[\d]{2})'
                        sales_from_string = re.search(sales_from_pattern, sales_from_string)[0]
                        item_dict['measure'] = measure_final
                        item_dict['salesStart'] = sales_from_string
                    else:
                        item_dict['measure'] = np.nan
                        
                    item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
                    item_dict['shopName'] = 'auchan'

                    if len(item_dict) != 0:
                        #print('adding item_dict')
                        all_items.append(item_dict)
            else:
                pass
                #print('prod is None')
                        
    df = pd.DataFrame(all_items)
    
    df = df.loc[df['price'] != '']
    
    return df

In [5]:
df = get_all_offer_auchan(all_link)

crawl url: https://www.auchan.hu/ajanlat/otthon-haztartas-kert-auto/mind/500 done 1 from 13
crawl url: https://www.auchan.hu/ajanlat/sport-jatek/mind/500 done 2 from 13
crawl url: https://www.auchan.hu/ajanlat/friss-elelmiszer/mind/500 done 3 from 13
crawl url: https://www.auchan.hu/ajanlat/italok/mind/500 done 4 from 13
crawl url: https://www.auchan.hu/ajanlat/fagyasztott-elelmiszerek/mind/500 done 5 from 13
crawl url: https://www.auchan.hu/ajanlat/tartos-elelmiszerek/mind/500 done 6 from 13
crawl url: https://www.auchan.hu/ajanlat/illatszer-szepsegapolas/mind/500 done 7 from 13
crawl url: https://www.auchan.hu/ajanlat/allatvilag/mind/500 done 8 from 13
crawl url: https://www.auchan.hu/ajanlat/vegyi-aruk-haztartas/mind/500 done 9 from 13
crawl url: https://www.auchan.hu/ajanlat/babavilag/mind/500 done 10 from 13
crawl url: https://www.auchan.hu/ajanlat/muszaki-cikkek/mind/500 done 11 from 13
crawl url: https://www.auchan.hu/ajanlat/ruhazat-1/mind/500 done 12 from 13
crawl url: https:/

In [6]:
df

,itemId,itemName,itemCleanName,price,source,measure,salesStart,runDate,shopName
0,NaN,ACTUEL CSÚSZÁSGÁTLÓ,actuel csuszasgatlo,1490 Ft,otthon-haztartas-kert-auto,Ft/db,2022.06.23,2022.06.23-20:56:09,auchan
1,NaN,ACTUEL MŰANYAG WC-ÜLŐKE,actuel muanyag wc-uloke,4490 Ft,otthon-haztartas-kert-auto,Ft/db,2022.06.23,2022.06.23-20:56:09,auchan
4,NaN,ACTUEl BÁRSONY STRAND- TÖRÖLKÖZŐ,actuel barsony strand- torolkozo,6990 Ft,otthon-haztartas-kert-auto,Ft/db,2022.06.23,2022.06.23-20:56:09,auchan
5,NaN,ACTUEL MICROFIBER TÖRÖLKÖZŐ,actuel microfiber torolkozo,1790 Ft,otthon-haztartas-kert-auto,Ft/db,2022.06.23,2022.06.23-20:56:09,auchan
6,NaN,ACTUEl BÁRSONY STRANDTÖRÖLKÖZŐ,actuel barsony strandtorolkozo,3690 Ft,otthon-haztartas-kert-auto,Ft/db,2022.06.23,2022.06.23-20:56:09,auchan
...,...,...,...,...,...,...,...,...,...
500,NaN,LG 43UP75003LF SMART LED TV,lg 43up75003lf smart led tv,129999 Ft,muszaki-cikkek,Ft/db,2022.06.23,2022.06.23-20:56:28,auchan
501,NaN,LG 55UP75003LF SMART LED TV,lg 55up75003lf smart led tv,164999 Ft,muszaki-cikkek,Ft/db,2022.06.23,2022.06.23-20:56:28,auchan
502,NaN,SELECLINE 50S211 SMART UHD TV,selecline 50s211 smart uhd tv,109990 Ft,muszaki-cikkek,Ft/db,2022.06.23,2022.06.23-20:56:28,auchan
503,NaN,FÉRFI SZÖRFNADRÁG,ferfi szorfnadrag,1690 Ft,ruhazat-1,Ft/db,2022.06.23,2022.06.23-20:56:30,auchan


In [7]:
df.to_excel('auchan_result.xlsx', index=False)

# backup

In [54]:
url = 'https://www.auchan.hu/ajanlat/otthon-haztartas-kert-auto/200'

In [55]:
page = urllib.request.urlopen(url)
soup = bs(page)

In [56]:
divs = soup.body.findAll('div', {'class' : ['row prodList']})

In [57]:
len(divs)

1

In [58]:
for div in divs:
    prods = div.findAll('div', {'class' : ['box p10px prodItem']})
    for prod in prods:
        print(prod.find('a', {'class' : ['title red']}).get_text())
        print(prod.find('strong', {'class' : ['red']}).get_text())
        print(prod.find('p').get_text().strip())

 ACTUEL CSÚSZÁSGÁTLÓ
1490 Ft
Kiszerelés:Ft/db
                2022.06.23-06.29.
 ACTUEL MŰANYAG WC-ÜLŐKE
4490 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 SZÁMOS ACTUEL FÜRDŐSZOBASZŐNYEG - 20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.
 SZÁMOS ACTUEL ZUHANYFÜGGÖNY - 20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.
 ACTUEl BÁRSONY STRAND- TÖRÖLKÖZŐ
6990 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEL MICROFIBER TÖRÖLKÖZŐ
1790 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEl BÁRSONY STRANDTÖRÖLKÖZŐ
3690 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEl STRANDMATRAC PÁRNÁVAL
6490 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEl ITALADAGOLÓ
999 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 SZÁMOS BOROS ÉS PEZSGŐS POHÁR -20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.
 SZÁMOS KANCSÓ -20 %

Kiszerelés:Ft/db
            2022.06.23-06.29.
 SZÁMOS TÁL   -20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.


In [ ]:
def get_all_link_auchan():
    
    all_link = []
    offer_page = 'https://www.auchan.hu/ajanlatok'
    for div in divs:
        for a in div.find_all('a', href=True):
            print(a['href'])
            all_link.append(a['href'] + '/500')
    
    return all_link